In [55]:
# Import required libraries
import os
import openai
import requests
import json
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment
OPENAI_API_KEY = os.getenv("genai_openai_key")
rapid_api_key = os.getenv("rapid_api_key")

# Set OpenAI API key
openai.api_key = OPENAI_API_KEY


# ---------------------------------------------------------------------------- #
# Step 1: Define the function that fetches weather data & Stock detail from RapidAPI          #
# ---------------------------------------------------------------------------- #

def get_current_weather(location):
    """Fetch the current weather for a given location using a 3rd party API"""

    url = "https://weather-api167.p.rapidapi.com/api/weather/forecast"
    querystring = {"place": location}

    headers = {
        'x-rapidapi-key': rapid_api_key,
        'x-rapidapi-host': "weather-api167.p.rapidapi.com",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()

def get_stock_details(stock_name):
    """Fetch the stock details for a given stock name or symbol using a 3rd party API"""

    url = "https://yahoo-finance166.p.rapidapi.com/api/news/list-by-symbol"
    querystring = {"s":stock_name}

    headers = {
	"x-rapidapi-key": rapid_api_key,
	"x-rapidapi-host": "yahoo-finance166.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    return response.json()


# ---------------------------------------------------------------------------- #
# Step 2: Define the function schema for OpenAI to understand this function   #
# ---------------------------------------------------------------------------- #

functions = [
    {
        "name": "get_current_weather",  # Must match the actual Python function name
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "get_stock_details",  # Must match the actual Python function name
        "description": "Get the stock details for the given symbol or name",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_name": {
                    "type": "string",
                    "description": "The stock name, e.g. TCS"
                }
            },
            "required": ["stock_name"]
        }
    }
]


# ---------------------------------------------------------------------------- #
# Step 3: Create a conversation with the user asking for weather              #
# ---------------------------------------------------------------------------- #

user_message1 = "Hi, There"
#user_message2 = "summarzie Kodaikanal, tamilnadu climate report"  # (typo, but GPT can still understand)
user_message2 = "Current market status of Gail"
# Prepare the message list for the chat completion
messages = []
messages.append({"role": "user", "content": user_message1})
messages.append({"role": "user", "content": user_message2})


# ---------------------------------------------------------------------------- #
# Step 4: Send messages and function schema to OpenAI                         #
# ---------------------------------------------------------------------------- #

response_1 = openai.chat.completions.create(
    model="gpt-3.5-turbo",       # Should ideally be "gpt-3.5-turbo-0613" for function calling
    messages=messages,           # Send the user's messages
    functions=functions          # Include function schema
)

# ---------------------------------------------------------------------------- #
# Step 5: Extract function call details from the response                     #
# ---------------------------------------------------------------------------- #

function_detail = response_1.choices[0].message.function_call

# Extract location argument from GPT's function call suggestion
function_name = function_detail.name  # Should be "get_current_weather"
arguments = eval(function_detail.arguments)
function_response = []
if function_name == "get_current_weather":
    response = get_current_weather(arguments["location"])  # Unsafe in general, use json.loads() instead
    for i in range(len(response['list'])):
        function_response.append(response['list'][i]['summery'])
    
elif function_name == "get_stock_details":
    response = get_stock_details(arguments['stock_name'])
    response_list = response['data']['main']['stream']
    for i in range(len(response_list)):
        function_response.append(response_list[i]['content']['title'])



# ---------------------------------------------------------------------------- #
# Step 6: Append the assistant's function call & our (dummy) function output  #
# ---------------------------------------------------------------------------- #

# Append GPT's message suggesting the function call
messages.append(response_1.choices[0].message)

# Instead of calling the actual function here, we simulate a placeholder
# In production, you'd call get_current_weather(location) and put its result as content
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": json.dumps(function_response)  # ❗ Should ideally be the **function output**, not just the location
    }
)


# ---------------------------------------------------------------------------- #
# Step 7: Send the updated message list to GPT for a final response           #
# ---------------------------------------------------------------------------- #

second_response = openai.chat.completions.create(
    model="gpt-3.5-turbo",  # Again, should be 0613 version for function calling support
    messages=messages
)

# Now GPT will return a final natural language message like:
# "The weather in Salem, Tamilnadu is light rain with 29°C."

print(second_response.choices[0].message.content)

I'm sorry, but I couldn't retrieve the current market status of Gail at the moment. You may check a financial news website or a stock market tracker for the most up-to-date information.
